tomemos valid_time como el eje temporal oficial, pero antes hagamos “autopsia preventiva” del archivo raro (1991_2025) para entender qué demonios representa ese time: 2.

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from pathlib import Path

FILES = {
    "1991_2024": Path("../historic/ERA5_T2M_monthly_1991_2024_chile.nc"),
    "1991_2025": Path("../historic/ERA5_T2M_monthly_1991_2025_chile.nc"),
    "2025_01_10": Path("../historic/ERA5_T2M_monthly_2025_01_10_chile.nc"),
}

for k,p in FILES.items():
    print(k, "exists=", p.exists(), "path=", p.resolve())


1991_2024 exists= True path= C:\Users\Asus\Documents\code\SbnAI\aws_meteo_backend\historic\ERA5_T2M_monthly_1991_2024_chile.nc
1991_2025 exists= True path= C:\Users\Asus\Documents\code\SbnAI\aws_meteo_backend\historic\ERA5_T2M_monthly_1991_2025_chile.nc
2025_01_10 exists= True path= C:\Users\Asus\Documents\code\SbnAI\aws_meteo_backend\historic\ERA5_T2M_monthly_2025_01_10_chile.nc


In [2]:
p = FILES["1991_2025"]
ds = xr.open_dataset(p, decode_cf=True, mask_and_scale=True)  # engine auto
ds


<xarray.Dataset> Size: 2MB
Dimensions:     (time: 2, valid_time: 37, latitude: 157, longitude: 41)
Coordinates:
  * time        (time) datetime64[ns] 16B 1970-01-01 1970-01-01
  * valid_time  (valid_time) datetime64[ns] 296B 2020-01-01 ... 2023-01-01
  * latitude    (latitude) float64 1kB -17.0 -17.25 -17.5 ... -55.5 -55.75 -56.0
  * longitude   (longitude) float64 328B 284.0 284.2 284.5 ... 293.5 293.8 294.0
    number      int64 8B ...
    expver      (time, valid_time) <U4 1kB ...
Data variables:
    t2m         (time, valid_time, latitude, longitude) float32 2MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-04T04:18 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
print("sizes:", dict(ds.sizes))
print("coords:", list(ds.coords))
print("data_vars:", list(ds.data_vars))
print("global attrs keys:", list(ds.attrs.keys())[:30])

for name in ["t2m", "time", "valid_time", "latitude", "longitude", "number", "expver"]:
    if name in ds:
        obj = ds[name]
        print(f"\n--- {name} ---")
        print("type:", type(obj))
        print("dims:", getattr(obj, "dims", None))
        print("shape:", getattr(obj, "shape", None))
        print("dtype:", getattr(obj, "dtype", None))
        if hasattr(obj, "attrs"):
            print("attrs keys:", list(obj.attrs.keys())[:25])


sizes: {'time': 2, 'valid_time': 37, 'latitude': 157, 'longitude': 41}
coords: ['valid_time', 'latitude', 'longitude', 'number', 'expver', 'time']
data_vars: ['t2m']
global attrs keys: ['GRIB_centre', 'GRIB_centreDescription', 'GRIB_subCentre', 'Conventions', 'institution', 'history']

--- t2m ---
type: <class 'xarray.core.dataarray.DataArray'>
dims: ('time', 'valid_time', 'latitude', 'longitude')
shape: (2, 37, 157, 41)
dtype: float32
attrs keys: ['GRIB_paramId', 'GRIB_dataType', 'GRIB_numberOfPoints', 'GRIB_typeOfLevel', 'GRIB_stepUnits', 'GRIB_stepType', 'GRIB_gridType', 'GRIB_uvRelativeToGrid', 'GRIB_NV', 'GRIB_Nx', 'GRIB_Ny', 'GRIB_cfName', 'GRIB_cfVarName', 'GRIB_gridDefinitionDescription', 'GRIB_iDirectionIncrementInDegrees', 'GRIB_iScansNegatively', 'GRIB_jDirectionIncrementInDegrees', 'GRIB_jPointsAreConsecutive', 'GRIB_jScansPositively', 'GRIB_latitudeOfFirstGridPointInDegrees', 'GRIB_latitudeOfLastGridPointInDegrees', 'GRIB_longitudeOfFirstGridPointInDegrees', 'GRIB_longitud

In [4]:
vt = ds["valid_time"]
print("valid_time.dims:", vt.dims)
print("valid_time.shape:", vt.shape)

# Muestra pocos valores sin reventar consola
vals = vt.values
print("first element type:", type(np.ravel(vals)[0]))
print("sample values:", np.ravel(vals)[:5], "...", np.ravel(vals)[-5:])


valid_time.dims: ('valid_time',)
valid_time.shape: (37,)
first element type: <class 'numpy.datetime64'>
sample values: ['2020-01-01T00:00:00.000000000' '2020-02-01T00:00:00.000000000'
 '2020-03-01T00:00:00.000000000' '2020-04-01T00:00:00.000000000'
 '2020-05-01T00:00:00.000000000'] ... ['2022-09-01T00:00:00.000000000' '2022-10-01T00:00:00.000000000'
 '2022-11-01T00:00:00.000000000' '2022-12-01T00:00:00.000000000'
 '2023-01-01T00:00:00.000000000']


Interacción eje y significado de time:2

In [5]:
t = ds["time"]
print("time.dims:", t.dims)
print("time.shape:", t.shape)
print("time dtype:", t.dtype)
print("time attrs:", t.attrs)

tv = t.values
print("time values:", tv)
print("time values types:", [type(x) for x in np.ravel(tv)])


time.dims: ('time',)
time.shape: (2,)
time dtype: datetime64[ns]
time attrs: {}
time values: ['1970-01-01T00:00:00.000000000' '1970-01-01T00:00:00.000000000']
time values types: [<class 'numpy.datetime64'>, <class 'numpy.datetime64'>]


In [6]:
da = ds["t2m"]
print("t2m.dims:", da.dims)
print("t2m.shape:", da.shape)
print("t2m attrs:", {k: da.attrs.get(k) for k in ["units", "standard_name", "long_name"] if k in da.attrs})

# muestra un mini slice numérico para confirmar que se lee
mini = da.isel(latitude=0, longitude=0)
print("mini dims:", mini.dims, "shape:", mini.shape)
print("mini values (first 10):", np.asarray(mini.values).ravel()[:10])


t2m.dims: ('time', 'valid_time', 'latitude', 'longitude')
t2m.shape: (2, 37, 157, 41)
t2m attrs: {'units': 'K', 'standard_name': 'unknown', 'long_name': '2 metre temperature'}
mini dims: ('time', 'valid_time') shape: (2, 37)
mini values (first 10): [295.53687 296.73828 296.90454 295.04004 292.81836 291.0691  289.29785
 288.7334  288.55127 289.52832]


In [7]:
def to_1d_datetime(arr):
    flat = np.ravel(arr)
    # intenta convertir a pandas datetime (si es cftime puede fallar)
    try:
        return pd.to_datetime(flat)
    except Exception as e:
        print("pd.to_datetime failed:", repr(e))
        return flat

# valid_time
vt_vals = ds["valid_time"].values
vt_flat = to_1d_datetime(vt_vals)

print("valid_time length (flat):", len(vt_flat))
try:
    s = pd.Series(vt_flat)
    print("valid_time duplicates:", int(s.duplicated().sum()))
    # monotonic check (solo si comparable)
    if hasattr(s.iloc[0], "to_datetime64") or np.issubdtype(s.dtype, np.datetime64):
        s_sorted = s.sort_values()
        print("valid_time min/max:", s_sorted.iloc[0], s_sorted.iloc[-1])
except Exception as e:
    print("valid_time analysis error:", repr(e))


valid_time length (flat): 37
valid_time duplicates: 0
valid_time min/max: 2020-01-01 00:00:00 2023-01-01 00:00:00


In [8]:
da = ds["t2m"]

if "time" in da.dims:
    # extrae series en el mismo punto para cada time index
    lat0 = int(ds.sizes["latitude"]//2)
    lon0 = int(ds.sizes["longitude"]//2)
    s0 = da.isel(time=0, latitude=lat0, longitude=lon0).values
    s1 = da.isel(time=1, latitude=lat0, longitude=lon0).values
    print("point idx:", (lat0, lon0))
    print("series lengths:", len(np.ravel(s0)), len(np.ravel(s1)))
    print("first 5 time=0:", np.ravel(s0)[:5])
    print("first 5 time=1:", np.ravel(s1)[:5])
    print("max abs diff:", float(np.nanmax(np.abs(np.ravel(s0)-np.ravel(s1)))))
else:
    print("t2m no tiene dim 'time' (ok).")


point idx: (78, 20)
series lengths: 37 37
first 5 time=0: [287.28687 285.44922 284.92603 280.69238 276.5    ]
first 5 time=1: [nan nan nan nan nan]
max abs diff: nan


C:\Users\Asus\AppData\Local\Temp\ipykernel_22816\2581797337.py:13: RuntimeWarning: All-NaN slice encountered
  print("max abs diff:", float(np.nanmax(np.abs(np.ravel(s0)-np.ravel(s1)))))


In [ ]:
def open_basic(path):
    return xr.open_dataset(path, decode_cf=True, mask_and_scale=True)

dsA = open_basic(FILES["1991_2024"])
dsB = open_basic(FILES["1991_2025"])
dsC = open_basic(FILES["2025_01_10"])

def grid_fingerprint(ds):
    lat = np.asarray(ds["latitude"].values).astype("float64")
    lon = np.asarray(ds["longitude"].values).astype("float64")
    return {
        "lat_shape": lat.shape,
        "lon_shape": lon.shape,
        "lat_minmax": (float(lat.min()), float(lat.max())),
        "lon_minmax": (float(lon.min()), float(lon.max())),
        "lat_step_med": float(np.median(np.abs(np.diff(lat)))),
        "lon_step_med": float(np.median(np.abs(np.diff(lon)))),
        "lat_first5": lat[:5],
        "lon_first5": lon[:5],
    }

for name, d in [("1991_2024", dsA), ("1991_2025", dsB), ("2025_01_10", dsC)]:
    fp = grid_fingerprint(d)
    print("\n==", name, "==")
    for k,v in fp.items():
        if isinstance(v, np.ndarray):
            print(k, ":", v)
        else:
            print(k, ":", v)

dsA.close(); dsB.close(); dsC.close()



== 1991_2024 ==
lat_shape : (157,)
lon_shape : (41,)
lat_minmax : (-56.0, -17.0)
lon_minmax : (-76.0, -66.0)
lat_step_med : 0.25
lon_step_med : 0.25
lat_first5 : [-17.   -17.25 -17.5  -17.75 -18.  ]
lon_first5 : [-76.   -75.75 -75.5  -75.25 -75.  ]

== 1991_2025 ==
lat_shape : (157,)
lon_shape : (41,)
lat_minmax : (-56.0, -17.0)
lon_minmax : (284.0, 294.0)
lat_step_med : 0.25
lon_step_med : 0.25
lat_first5 : [-17.   -17.25 -17.5  -17.75 -18.  ]
lon_first5 : [284.   284.25 284.5  284.75 285.  ]

== 2025_01_10 ==
lat_shape : (157,)
lon_shape : (41,)
lat_minmax : (-56.0, -17.0)
lon_minmax : (-76.0, -66.0)
lat_step_med : 0.25
lon_step_med : 0.25
lat_first5 : [-17.   -17.25 -17.5  -17.75 -18.  ]
lon_first5 : [-76.   -75.75 -75.5  -75.25 -75.  ]


: 

In [ ]:
def summarize_time(path):
    ds = xr.open_dataset(path, decode_cf=True, mask_and_scale=True)
    key = "valid_time" if "valid_time" in ds.coords else ("time" if "time" in ds.coords else None)
    if key is None:
        print(path.name, "NO TIME COORD")
        ds.close()
        return
    arr = ds[key].values
    flat = np.ravel(arr)
    try:
        flat_dt = pd.to_datetime(flat)
        print(path.name, key, "len=", len(flat_dt), "min=", flat_dt.min(), "max=", flat_dt.max())
    except Exception as e:
        print(path.name, key, "len=", len(flat), "type=", type(flat[0]), "error=", repr(e))
    ds.close()

for k,p in FILES.items():
    summarize_time(p)


ERA5_T2M_monthly_1991_2024_chile.nc valid_time len= 36 min= 2020-01-01 00:00:00 max= 2022-12-01 00:00:00
